In [1]:
import at
import sys
from at import Param, ParamArray
if sys.version_info.minor < 9:
    from importlib_resources import files, as_file
else:
    from importlib.resources import files, as_file

In [2]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    ring = at.load_lattice(path)

# Parameters

Parameters are objects of class {py:class}`.Param` which can be used instead of numeric values as {py:class}`.Element` attributes.

Parameters are initialised with a **scalar** numeric value. They have an optional name, only used to identify them in printed output:

In [3]:
p1=Param(2.5, name='total length')
p2=Param(1.0)
print(p1, p2)

Param(2.5, name='total length') Param(1.0, name='param2')


The value of a parameter can be read or modified through its {py:attr}`~.variables.Variable.value` property. {py:meth}`~.variables.Variable.set` and {py:meth}`~.variables.Variable.get` methods are also available:

In [4]:
print(p1.value)
p1.value = 2.4
print(p1)
p1.set(2.3)
p1.get()

2.5
Param(2.4, name='total length')


2.3

Arithmetic combinations of parameters create new read-only parameters of class {py:class}`.ParamBase`, whose value is permanently kept up-to-date:

In [5]:
p3 = p1-p2
print(p3)
p2.value = 0.9
print(p3)

ParamBase(1.2999999999999998, name='calc1')
ParamBase(1.4, name='calc1')


Parameters may be assigned to {py:class}`.Element` attributes, for instance on initialisation:

In [6]:
dr1 = at.Drift('DR1', p2)
qf1 = at.Quadrupole('QF1', p3, 0.6)
print(dr1)
print(qf1)

Drift:
	FamName : DR1
	Length : Param(0.9, name='param2')
	PassMethod : DriftPass
Quadrupole:
	FamName : QF1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.  0.6]
	K : 0.6


The {py:class}`.Element` attributes keep their type so that all the processing of elements either in python functions or in C integrators is unchanged:

In [7]:
print(dr1.Length, type(dr1.Length))

0.9 <class 'float'>


## Assigning parameters

### To a single element
Parameters may be assigned to {py:class}`.Element` attributes in several ways:

**At element creation:**

In [8]:
dr2 = at.Drift('DR2', p2)

**By converting a numeric attribute into a parameter:**

In [9]:
qd1 = at.Quadrupole('QD1', 0.5, -0.4)
p4 = qd1.parametrise('Length')
print(p4)

Param(0.5, name='param3')


**By normal assignment:**

In [10]:
qd1.Length = p3
print(qd1)

Quadrupole:
	FamName : QD1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [ 0.  -0.4]
	K : -0.4


**With the {py:meth}`~.Element.set_parameter` method:**

In [11]:
qd1.set_parameter('Length', p4)
print(qd1)

Quadrupole:
	FamName : QD1
	Length : Param(0.5, name='param3')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [ 0.  -0.4]
	K : -0.4


### To selected elements of a {py:class}`.Lattice`
To act on several elements in a single step, {py:class}`.Lattice` methods similar to {py:class}`.Element` methods are available

**Convert numeric attributes into parameters:**

The attribute of all the selected elements is replaced by a single parameter whose initial value is the average of the original values.

In [12]:
p5 = ring.parametrise('QF1[AE]', 'PolynomB', index=1, name='kf1')
print(p5)
print(ring[5])

Param(2.5394599781303304, name='kf1')
Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, Param(2.5394599781303304, name='kf1')])
	K : Param(2.5394599781303304, name='kf1')


**Use the {py:meth}`~.Lattice.set_parameter` method:**

The attribute of all the selected elements is replaced by the provided parameter,

In [13]:
p6 = Param(0.311896, name='lf1')
ring.set_parameter('QF1[AE]', 'Length', p6)
print(ring[117])

Quadrupole:
	FamName : QF1E
	Length : Param(0.311896, name='lf1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, Param(2.5394599781303304, name='kf1')])
	K : Param(2.5394599781303304, name='kf1')


### Special case of array attributes

Normal {py:class}`.Element` array attributes are numeric numpy arrays, so they cannot be assigned objects. Attempting to assign a parameter to an item of an array attribute will assign the current parameter value instead of the parameter itself. Since this is not what is usually desired, a warning is emitted:

In [14]:
p5 = Param(-0.3, name='QD strength')
qd1.PolynomB[1] = p5

/Users/famille/dev/libraries/at/pyat/at/lattice/elements.py:30: UserWarning: 

The Parameter 'QD strength' is ignored, instead its value '-0.3' is used.
To set a parameter in an array, you must first parametrise the array itself.

  warn(UserWarning(message))


To accept parameters, the array attribute itself must be first replaced by a {py:class}`.ParamArray` object. The easiest way for that is to let the {py:meth}`~.Element.parametrise` or {py:meth}`~.Element.set_parameter` methods take care of it:

In [15]:
qd1.set_parameter('PolynomB', p5, index=1)
print(qd1)

Quadrupole:
	FamName : QD1
	Length : Param(0.5, name='param3')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, Param(-0.3, name='QD strength')])
	K : Param(-0.3, name='QD strength')


Once the array attribute is a {py:class}`.ParamArray`, any of its items may be assigned a parameter. As with scalar attributes, the type of the attribute is unchanged.

Another possiblility is to first convert the array attribute into a {py:class}`.ParamArray`, and then perform a standard assignment:

In [16]:
qd1 = at.Quadrupole('QD1', p3, -0.4)
qd1.parametrise('PolynomB')
qd1.PolynomB[1] = p5
print(qd1)

Quadrupole:
	FamName : QD1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, Param(-0.3, name='QD strength')])
	K : Param(-0.3, name='QD strength')


## Retrieving parameters

Since the values of {py:class}`.Element` attributes keep their original type, they cannot be used to access the underlying parameter. The only way to retrieve it is to use the {py:meth}`~.Element.get_parameter` method. When applied to a non-parameter attribute, the attribute is returned.

In [17]:
pp = qd1.get_parameter('Length')
print(pp)
print("pp is p3:", pp is p3)

ParamBase(1.4, name='calc1')
pp is p3: True


This also works for items in array attributes:

In [18]:
pp = qd1.get_parameter('PolynomB', index=1)
print(pp)
print("pp is p5:", pp is p5)

Param(-0.3, name='QD strength')
pp is p5: True


Retrieving the whole array attribute returns a {py:class}`.ParamArray` object. It may contain either numbers or {py:class}`.Param` objects:

In [19]:
pp = qd1.get_parameter('PolynomB')
print(pp)

ParamArray([0.0, Param(-0.3, name='QD strength')])


## Checking parametrisation

The {py:meth}`~.Element.is_parametrised` method may be applied to:
- a full element: it returns true is any of its attributes is a parameter,
- an array attribute: it returns true if any of its items is a parameter,
- a scalar attribute or an item of an array.

In [20]:
print(qd1.is_parametrised())
print(qd1.is_parametrised('PolynomB'))
print(qd1.is_parametrised('PolynomA'))
print(qd1.is_parametrised('PolynomB', index=1))
print(qd1.is_parametrised('PolynomB', index=0))

True
True
False
True
False


## Removing parameters
Removing the parameters will "freeze" the element at its current value. The {py:meth}`~.Element.unparametrise` method is defined for both {py:class}`.Element` and {py:class}`.Lattice`, and may be applied to:
- a full element: all the parameters are replaced by their value,
- an array attribute: the whole {py:class}`.ParamArray` is replaced by a numpy array,
- a scalar attribute or an item of an array.

### In an Element

In [21]:
print(qd1)
qd1.unparametrise('Length')
qd1.unparametrise('PolynomB', 1)
print(qd1)
qd1.unparametrise()
print(qd1)

Quadrupole:
	FamName : QD1
	Length : ParamBase(1.4, name='calc1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, Param(-0.3, name='QD strength')])
	K : Param(-0.3, name='QD strength')
Quadrupole:
	FamName : QD1
	Length : 1.4
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, -0.3])
	K : -0.3
Quadrupole:
	FamName : QD1
	Length : 1.4
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 10
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [ 0.  -0.3]
	K : -0.3


### In a Lattice

In [22]:
print(ring[5])
ring.unparametrise('QF1[AE]', 'Length')
ring.unparametrise('QF1[AE]', 'PolynomB', index=1)
print(ring[5])
ring.unparametrise('QF1[AE]')
print(ring[117])

Quadrupole:
	FamName : QF1A
	Length : Param(0.311896, name='lf1')
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, Param(2.5394599781303304, name='kf1')])
	K : Param(2.5394599781303304, name='kf1')
Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : ParamArray([0.0, 2.5394599781303304])
	K : 2.5394599781303304
Quadrupole:
	FamName : QF1E
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.         2.53945998]
	K : 2.5394599781303304


## Parameter history
Parameter values are kept in an {py:attr}`~.variables.Variable.history` buffer. The properties {py:attr}`~.variables.Variable.initial_value`, {py:attr}`~.variables.Variable.last_value` and {py:attr}`~.variables.Variable.previous_value` are also available:

In [23]:
p1.value = 2.2
print(p1.history)
print(p1.initial_value)
print(p1.previous_value)

[2.5, 2.4, 2.3, 2.2]
2.5
2.3


After varying parameters, in matching for instance, the current status can be printed:

In [24]:
print(p1.status())


        Name      Initial          Final        Variation

total length    2.500000e+00    2.200000e+00   -3.000000e-01


Parameters may be reset to a previous history value with the {py:meth}`~.variables.Variable.set_initial` and {py:meth}`~.variables.Variable.set_previous` methods. The history is shortened accordingly.

## Copying and Saving

In a shallow copy of an {py:class}`.Element`, the parameter attributes are shared between the {py:class}`.Element` and its copy.

In a deep copy of an {py:class}`.Element`, a copy of the original parameter is set in the {py:class}`.Element` copy.

When saving a lattice with parametrised elements, as a `.mat`, a `.m` or `.repr` file, all parametrisation is removed, and a "frozen" state of the lattice is saved.

In pickle dumps of an {py:class}`.Element`, parameters are preserved.

In [25]:
p1.set_previous()
print(p1, p1.history)
p1.set_initial()
print(p1, p1.history)

Param(2.3, name='total length') [2.5, 2.4, 2.3]
Param(2.5, name='total length') [2.5]
